__File: src_physics_write_data.ipynb__

__Author:   A. J. Tropiano (tropiano.4@osu.edu)__<br/>
__Date:     Apr 09, 2021__

Writes SRC physics data files for plotting purposes. Use these files with src_physics_fig.ipynb to generate figures relying only data files.

__Revision history:<br/>__
* Apr 13, 2021 - Finished the following functions: n_lambda_deuteron_decomposition, deuteron_partial_wave_contributions, srg_transformation_partial_wave_ratios, lda_pair_momentum_distributions, lda_single_nucleon_momentum_distributions, lda_deuteron_pair_momentum_distribution, lda_deuteron_single_nucleon_momentum_distribution, a2_scaling_factors, and deuteron_partial_wave_ratios.
* Apr 16, 2021 - Adding Fe56 to LDA data files.
* Apr 17, 2021 - Generating data for Gezerlis N2LO 1 and 1.2 fm potentials.
* Apr 18, 2021 - Adding option to save directly to Dropbox directory in addition to LENT repository.
* Apr 21, 2021 - Generating momentum distributions at $\lambda=1.5$ and $2$ fm$^{-1}$.
* Apr 28, 2021 - Generating data files for new momentum mesh with $k_{\rm max}=15$ fm$^{-1}$.
* May 07, 2021 - Added snmd_partial_wave_contributions function to test S-wave dominance.
* May 11, 2021 - Added factorization functions under Extras.
* May 27, 2021 - Updating data files after correcting integrations in pmd.py and snmd.py.

In [1]:
from os import chdir, getcwd
import numpy as np
import shutil
import time
# Scripts made by A.T.
from dmd import deuteron_momentum_distributions
from Figures import figures_functions as ff
from lda import load_density, LDA
import observables as ob
import operators as op
from pmd import pair_momentum_distributions
from Potentials.vsrg_macos import vnn
from snmd import single_nucleon_momentum_distributions
from SRG.srg_unitary_transformation import SRG_unitary_transformation

__Set-up__

In [2]:
# Current working directory
cwd = getcwd()

# Specify potential
kvnn = 6 # AV18
# kvnn = 222 # Gezerlis N2LO 1 fm
# kvnn = 224 # Gezerlis N2LO 1.2 fm

# Save data in the following directory
data_directory = 'Figures/SRC_physics/Data/kvnn_%d' % kvnn

# Option to also save files to Dropbox directory
dropbox_directory = '../../../Dropbox/SRG_projects/Operator_evolution/Figures/SRC_physics/Data/kvnn_%d' % kvnn
save_to_dropbox = False
# save_to_dropbox = True

# Specify which momentum mesh you want to use
# kmax, kmid, ntot = 10.0, 2.0, 120 # Mesh artifacts seen in last few points of 3S1-3D1 \delta U for AV18
kmax, kmid, ntot = 15.0, 3.0, 120

# SRG evolve to 1.35 fm^-1
lamb = 1.35 

# Channels to include in calculation of momentum distributions
channels = ('1S0', '3S1') # S-waves only
# channels = ('1S0', '3S1', '3P0', '1P1', '3P1') # Still having issue with '3P2' and '3D3'

# Nuclei to show
nuclei = ( ('C12', 6, 6), ('O16', 8, 8), ('Ca40', 20, 20), ('Ca48', 20, 28), ('Fe56', 26, 30),
           ('Pb208', 82, 126) )

__Decomposition of $n^{\lambda}_d(q)$__

In [3]:
def n_lambda_deuteron_decomposition(kvnn, lamb, kmax=10.0, kmid=2.0, ntot=120, save_to_dropbox=False):
    """
    SRG-evolved deuteron pair momentum distribution as calculated by our expansion of U(k, k') in second
    quantization using deuteron wave function (not LDA). Gives each of the contributions from 1, \delta U,
    \delta U \delta U^\dagger.
    
    Parameters
    ----------
    kvnn : int
        This number specifies the potential.
    lamb : float
        SRG \lambda parameter [fm^-1].
    kmax : float, optional
        Maximum value in the momentum mesh [fm^-1].
    kmid : float, optional
        Mid-point value in the momentum mesh [fm^-1].
    ntot : int, optional
        Number of momentum points in mesh.
    save_to_dropbox : bool, optional
        Option to save file to Dropbox directory.
    
    """

    # Make file name
    file_name = 'n_lambda_deuteron_decomposition_kvnn_%d_lamb_%.2f_kmax_%.1f' % (kvnn, lamb, kmax)
    file_name = ff.replace_periods(file_name) + '.dat'

    # Load momentum
    q_array, _ = vnn.load_momentum(kvnn, '3S1', kmax, kmid, ntot)
    
    # Initialize deuteron momentum distributions class for given potential
    dmd = deuteron_momentum_distributions(kvnn, lamb, kmax, kmid, ntot)

    # Open file and write header where we allocate roughly 18 centered spaces for each label
    f = open(data_directory + '/' + file_name, 'w')
    header = '#' + '{:^17s}{:^18s}{:^18s}{:^18s}{:^18s}'.format('q', 'total', '1 term', '\delta U term',
                                                                '\delta U^2 term')
    f.write(header + '\n')
    
    # Loop over momenta q
    for iq, q in enumerate(q_array):
        
        # Calculate each contribution to n_\lambda(q) including total
        total, term_1, term_deltaU, term_deltaU2 = dmd.n_lambda_pair_exact(q, 'q_contributions')

        # Write to data file following the format from the header
        line = '{:^18.6f}{:^18.6e}{:^18.6e}{:^18.6e}{:^18.6e}'.format(q, total, term_1, term_deltaU,
                                                                      term_deltaU2)
        f.write('\n' + line)

    # Close file
    f.close()
    
    # Save to Dropbox?
    if save_to_dropbox:
        _ = shutil.copy(data_directory + '/' + file_name, dropbox_directory)

In [4]:
# # Contributions to deuteron momentum distribution in terms of 1, \delta U, and \delta U^2

# t0 = time.time() # Start time
# n_lambda_deuteron_decomposition(kvnn, lamb, kmax, kmid, ntot, save_to_dropbox)
# t1 = time.time() # Stop time
# mins = round( (t1 - t0) / 60.0, 5) # Minutes elapsed
# print('%.5f minutes elasped' % mins)

__Deuteron partial wave contributions__

In [5]:
def deuteron_partial_wave_contributions(kvnn, lamb, kmax=10.0, kmid=2.0, ntot=120, save_to_dropbox=False):
    """
    Contributions of each partial wave coupled-channel to |\psi_d(q)|^2 comparing unevolved and SRG-evolved
    for various momenta q [fm^-1].
    
    Parameters
    ----------
    kvnn : int
        This number specifies the potential.
    lamb : float
        SRG \lambda parameter [fm^-1].
    kmax : float, optional
        Maximum value in the momentum mesh [fm^-1].
    kmid : float, optional
        Mid-point value in the momentum mesh [fm^-1].
    ntot : int, optional
        Number of momentum points in mesh.
    save_to_dropbox : bool, optional
        Option to save file to Dropbox directory.
    
    """

    # Make file name
    file_name = 'deuteron_partial_wave_contributions_kvnn_%d_lamb_%.2f_kmax_%.1f' % (kvnn, lamb, kmax)
    file_name = ff.replace_periods(file_name) + '.dat'

    # Channel is 3S1-3D1 for deuteron
    channel = '3S1'
    
    # Load momentum [fm^-1]
    q_array, q_weights = vnn.load_momentum(kvnn, channel, kmax, kmid, ntot)
    
    # Load initial and evolved Hamiltonians [MeV]
    H_initial = vnn.load_hamiltonian(kvnn, channel, kmax, kmid, ntot)
    H_evolved = vnn.load_hamiltonian(kvnn, channel, kmax, kmid, ntot, method='srg', generator='Wegner',
                                     lamb=lamb)
    
    # Calculate SRG transformation [unitless]
    U_matrix = SRG_unitary_transformation(H_initial, H_evolved)

    # Load initial and evolved wave functions [unitless]
    psi_initial = ob.wave_function(H_initial)
    psi_evolved = ob.wave_function(H_initial, U=U_matrix)

    # Open file and write header where we allocate roughly 18 centered spaces for each label
    f = open(data_directory + '/' + file_name, 'w')
    header = '#' + '{:^17s}{:^18s}{:^18s}{:^18s}{:^18s}{:^18s}{:^18s}'.format('q', '3S1-3S1 init.',
             '3S1-3D1 init.', '3D1-3D1 init.', '3S1-3S1 SRG', '3S1-3D1 SRG', '3D1-3D1 SRG')
    f.write(header + '\n')

    # Loop over momenta q
    for iq, q in enumerate(q_array):
        
        # Initial and evolved momentum projection operators
        operator_initial = op.momentum_projection_operator(q, q_array, q_weights, channel, smeared=False)
        operator_evolved = op.momentum_projection_operator(q, q_array, q_weights, channel, U_matrix,
                                                           smeared=False)
    
        # Each contribution to |\psi_d(q)|^2 for initial with total
        psi2_3S1_3S1_initial = abs( psi_initial.T[:ntot] @ operator_initial[:ntot, :ntot] @ psi_initial[:ntot] )
        psi2_3S1_3D1_initial = abs( psi_initial.T[:ntot] @ operator_initial[:ntot, ntot:] @ psi_initial[ntot:] )
        psi2_3D1_3S1_initial = abs( psi_initial.T[ntot:] @ operator_initial[ntot:, :ntot] @ psi_initial[:ntot] )
        psi2_3D1_3D1_initial = abs( psi_initial.T[ntot:] @ operator_initial[ntot:, ntot:] @ psi_initial[ntot:] )
        psi2_initial = psi2_3S1_3S1_initial + psi2_3S1_3D1_initial + psi2_3D1_3S1_initial + psi2_3D1_3D1_initial

        # Each contribution to |\psi_d(q)|^2 for evolved with total
        psi2_3S1_3S1_evolved = abs( psi_evolved.T[:ntot] @ operator_evolved[:ntot, :ntot] @ psi_evolved[:ntot] )
        psi2_3S1_3D1_evolved = abs( psi_evolved.T[:ntot] @ operator_evolved[:ntot, ntot:] @ psi_evolved[ntot:] )
        psi2_3D1_3S1_evolved = abs( psi_evolved.T[ntot:] @ operator_evolved[ntot:, :ntot] @ psi_evolved[:ntot] )
        psi2_3D1_3D1_evolved = abs( psi_evolved.T[ntot:] @ operator_evolved[ntot:, ntot:] @ psi_evolved[ntot:] )
        psi2_evolved = psi2_3S1_3S1_evolved + psi2_3S1_3D1_evolved + psi2_3D1_3S1_evolved + psi2_3D1_3D1_evolved
        
        # Calculate the percentage of each contribution
        cont_3S1_3S1_initial = psi2_3S1_3S1_initial / psi2_initial * 100
        cont_3S1_3D1_initial = psi2_3S1_3D1_initial / psi2_initial * 100 * 2 # Factor of 2 for 3D1-3S1
        cont_3D1_3D1_initial = psi2_3D1_3D1_initial / psi2_initial * 100
        cont_3S1_3S1_evolved = psi2_3S1_3S1_evolved / psi2_evolved * 100
        cont_3S1_3D1_evolved = psi2_3S1_3D1_evolved / psi2_evolved * 100 * 2 # Factor of 2 for 3D1-3S1
        cont_3D1_3D1_evolved = psi2_3D1_3D1_evolved / psi2_evolved * 100
        
        # Write to data file following the format from the header
        line = '{:^18.6f}{:^18.6f}{:^18.6f}{:^18.6f}{:^18.6f}{:^18.6f}{:^18.6f}'.format(q, cont_3S1_3S1_initial,
                cont_3S1_3D1_initial, cont_3D1_3D1_initial, cont_3S1_3S1_evolved, cont_3S1_3D1_evolved,
                cont_3D1_3D1_evolved)
        f.write('\n' + line)

    # Close file
    f.close()
    
    # Save to Dropbox?
    if save_to_dropbox:
        _ = shutil.copy(data_directory + '/' + file_name, dropbox_directory)

In [6]:
# # Deuteron partial wave contributions |\psi_d(q)|^2

# t0 = time.time() # Start time
# deuteron_partial_wave_contributions(kvnn, lamb, kmax, kmid, ntot, save_to_dropbox)
# t1 = time.time() # Stop time
# mins = round( (t1 - t0) / 60.0, 5) # Minutes elapsed
# print('%.5f minutes elasped' % mins)

__($^{3}S_1-^{3}D_1$) / $^{1}S_0$ SRG transformation ratio__

In [7]:
def srg_transformation_partial_wave_ratios(k_0, kvnn, lamb, kmax=10.0, kmid=2.0, ntot=120,
                                           save_to_dropbox=False):
    """
    Ratio of |U(k_0, q)_{3S1-3D1}|^2 / |U(k_0, q)_{1S0}|^2 where k_0 < \lambda.
    
    Parameters
    ----------
    k_0 : float
        k_0 value in the ratio U(k_0, q)_{3S1} / U(k_0, q)_{1S0} [fm^-1].
    kvnn : int
        This number specifies the potential.
    lamb : float
        SRG \lambda parameter [fm^-1].
    kmax : float, optional
        Maximum value in the momentum mesh [fm^-1].
    kmid : float, optional
        Mid-point value in the momentum mesh [fm^-1].
    ntot : int, optional
        Number of momentum points in mesh.
    save_to_dropbox : bool, optional
        Option to save file to Dropbox directory.

    """

    # Make file name
    file_name = 'srg_transformation_partial_wave_ratios_k_0_%.1f_kvnn_%d_lamb_%.2f_kmax_%.1f' % (k_0, kvnn,
                                                                                                 lamb, kmax)
    file_name = ff.replace_periods(file_name) + '.dat'

    # Load momentum and weights (channel argument doesn't matter here)
    k_array, k_weights = vnn.load_momentum(kvnn, '1S0')
    
    # Divide out integration factors with factor_array
    factor_array = np.sqrt( (2*k_weights) / np.pi ) * k_array
    row, col = np.meshgrid(factor_array, factor_array)
    # Same thing but for coupled-channel
    factor_array_cc = np.concatenate( (factor_array, factor_array) )
    row_cc, col_cc = np.meshgrid(factor_array_cc, factor_array_cc)
    
    # Load initial and evolved Hamiltonians [MeV], and SRG transformations
    # 1S0
    H_initial_1s0 = vnn.load_hamiltonian(kvnn, '1S0')
    H_evolved_1s0 = vnn.load_hamiltonian(kvnn, '1S0', method='srg', lamb=lamb)
    # 3S1
    H_initial_3s1 = vnn.load_hamiltonian(kvnn, '3S1')
    H_evolved_3s1 = vnn.load_hamiltonian(kvnn, '3S1', method='srg', lamb=lamb)
    
    # Calculate SRG transformation with eigenvectors of initial and evolved Hamiltonians dividing out 
    # integration factors
    U_matrix_1s0 = SRG_unitary_transformation(H_initial_1s0, H_evolved_1s0) / row / col
    U_matrix_3s1 = SRG_unitary_transformation(H_initial_3s1, H_evolved_3s1) / row_cc / col_cc
    
    # Index of k_0 in k_array
    k_0_index = op.find_q_index(k_0, k_array)
        
    # Calculate |U(k_0, q)_{3S1-3D1}|^2
    numerator_array = U_matrix_3s1[k_0_index, :ntot] * U_matrix_3s1.T[:ntot, k_0_index] + \
                      U_matrix_3s1[k_0_index, ntot:] * U_matrix_3s1.T[ntot:, k_0_index]
        
    # Calculate |U(k, q_i)_{1S0}|^2
    denominator_array = U_matrix_1s0[k_0_index, :ntot] * U_matrix_1s0.T[:ntot, k_0_index]
        
    # Take ratio of 3S1-3D1 / 1S0
    ratio_array = numerator_array / denominator_array
    
    # Open file and write header where we allocate roughly 18 centered spaces for each label
    f = open(data_directory + '/' + file_name, 'w')
    header = '#' + '{:^17s}{:^18s}'.format('k', 'ratio')
    f.write(header + '\n')
    
    # Loop over momenta k
    for ik, k in enumerate(k_array):

        # Write to data file following the format from the header
        line = '{:^18.6f}{:^18.6e}'.format(k, ratio_array[ik])
        f.write('\n' + line)

    # Close file
    f.close()
    
    # Save to Dropbox?
    if save_to_dropbox:
        _ = shutil.copy(data_directory + '/' + file_name, dropbox_directory)

In [8]:
# # U_{3S1-3D1} / U_{1S0} ratios with k_0=0.1 fm^-1

# k_0 = 0.1

# t0 = time.time() # Start time
# srg_transformation_partial_wave_ratios(k_0, kvnn, lamb, kmax, kmid, ntot, save_to_dropbox)
# t1 = time.time() # Stop time
# mins = round( (t1 - t0) / 60.0, 5) # Minutes elapsed
# print('%.5f minutes elasped' % mins)

__Pair momentum distributions $n_{\lambda}^A(q,Q=0)$ under LDA__

In [9]:
def lda_pair_momentum_distributions(nucleus, channels, kvnn, lamb, kmax=10.0, kmid=2.0, ntot=120,
                                    save_to_dropbox=False):
    """
    Nuclear-averaged and SRG-evolved pair momentum distributions n_\lambda^A(q,Q=0).
    
    Parameters
    ----------
    nucleus : tuple
        Details for various nuclei formatted as a tuple: (name (str), Z (int), N (int)) (e.g., ('O16', 8, 8)).
    channels : tuple
        Partial wave channels to include in the calculation (e.g., ('1S0', '3S1')).
    kvnn : int
        This number specifies the potential.
    lamb : float
        SRG \lambda parameter [fm^-1].
    kmax : float, optional
        Maximum value in the momentum mesh [fm^-1].
    kmid : float, optional
        Mid-point value in the momentum mesh [fm^-1].
    ntot : int, optional
        Number of momentum points in mesh.
    save_to_dropbox : bool, optional
        Option to save file to Dropbox directory.
        
    Notes
    -----
    Looks like there is a bug for first q-value with S-, P-, and D-waves (not there when we take only S).
        
    """

    # Name of nucleus, proton number, neutron number, and mass number
    nucleus_name = nucleus[0]
    Z = nucleus[1]
    N = nucleus[2]
    A = N + Z
    
    # Make file name
    file_name = 'lda_pmd_%s_channels' % nucleus_name
    # Add each channel to file name
    for channel in channels:
        file_name += '_%s' % channel
    file_name += '_kvnn_%d_lamb_%.2f_kmax_%.1f' % (kvnn, lamb, kmax)
    file_name = ff.replace_periods(file_name) + '.dat'

    # Load momentum (channel argument doesn't matter here)
    q_array, _ = vnn.load_momentum(kvnn, '3S1', kmax, kmid, ntot)
    
    # Initialize pair momentum distributions class for given potential
    pmd = pair_momentum_distributions(kvnn, channels, lamb, kmax, kmid, ntot)
    
    # Load r values and nucleonic densities (the r_array's are the same)
    r_array, rho_p_array = load_density(nucleus_name, 'proton', Z, N)
    r_array, rho_n_array = load_density(nucleus_name, 'neutron', Z, N)
    
    # Call LDA class
    lda = LDA(r_array, rho_p_array, rho_n_array)
    
    # Calculate nuclear-averaged momentum distributions
    n_pp_array = lda.local_density_approximation(q_array, pmd.n_lambda_pp, 'pp')
    n_pn_array = lda.local_density_approximation(q_array, pmd.n_lambda_pn, 'pn')
    # np is the same as pn (make sure you still account for np in figures, that is,
    # you will need to multiply pn by 2)
    n_nn_array = lda.local_density_approximation(q_array, pmd.n_lambda_pp, 'nn')
    
    # Open file and write header where we allocate roughly 18 centered spaces for each label
    f = open(data_directory + '/' + file_name, 'w')
    header = '#' + '{:^17s}{:^18s}{:^18s}{:^18s}'.format('q', 'pp', 'pn', 'nn')
    f.write(header + '\n')
    
    # Loop over momenta q
    for iq, q in enumerate(q_array):

        # Write to data file following the format from the header
        line = '{:^18.6f}{:^18.6e}{:^18.6e}{:^18.6e}'.format(q, n_pp_array[iq], n_pn_array[iq], n_nn_array[iq])
        f.write('\n' + line)

    # Close file
    f.close()
    
    # Save to Dropbox?
    if save_to_dropbox:
        _ = shutil.copy(data_directory + '/' + file_name, dropbox_directory)

In [10]:
# # LDA pair momentum distributions

# t0 = time.time() # Start time
# # Loop over nuclei
# for inucleus in nuclei:
#     lda_pair_momentum_distributions(inucleus, channels, kvnn, lamb, kmax, kmid, ntot, save_to_dropbox)
# t1 = time.time() # Stop time
# mins = round( (t1 - t0) / 60.0, 5) # Minutes elapsed
# print('%.5f minutes elasped' % mins)

In [11]:
# # LDA pair momentum distributions with \lambda=1.5, 2, and 6 fm^-1

# lambdas = (1.5, 2.0, 6.0)

# t0 = time.time() # Start time
# # Loop over nuclei
# for inucleus in nuclei:
#     for ilamb in lambdas:
#         lda_pair_momentum_distributions(inucleus, channels, kvnn, ilamb, kmax, kmid, ntot, save_to_dropbox)
# t1 = time.time() # Stop time
# mins = round( (t1 - t0) / 60.0, 5) # Minutes elapsed
# print('%.5f minutes elasped' % mins)

__Single-nucleon momentum distributions $n_{\lambda}^A(q)$ under LDA__

In [12]:
def lda_single_nucleon_momentum_distributions(nucleus, channels, kvnn, lamb, kmax=10.0, kmid=2.0, ntot=120,
                                              save_to_dropbox=False):
    """
    Nuclear-averaged and SRG-evolved single-nucleon momentum distributions n_\lambda^A(q).
    
    Parameters
    ----------
    nucleus : tuple
        Details for various nuclei formatted as a tuple: (name (str), Z (int), N (int)) (e.g., ('O16', 8, 8)).
    channels : tuple
        Partial wave channels to include in the calculation (e.g., ('1S0', '3S1')).
    kvnn : int
        This number specifies the potential.
    lamb : float
        SRG \lambda parameter [fm^-1].
    kmax : float, optional
        Maximum value in the momentum mesh [fm^-1].
    kmid : float, optional
        Mid-point value in the momentum mesh [fm^-1].
    ntot : int, optional
        Number of momentum points in mesh.
    save_to_dropbox : bool, optional
        Option to save file to Dropbox directory.
        
    """

    # Name of nucleus, proton number, neutron number, and mass number
    nucleus_name = nucleus[0]
    Z = nucleus[1]
    N = nucleus[2]
    A = N + Z
    
    # Make file name
    file_name = 'lda_snmd_%s_channels' % nucleus_name
    # Add each channel to file name
    for channel in channels:
        file_name += '_%s' % channel
    file_name += '_kvnn_%d_lamb_%.2f_kmax_%.1f' % (kvnn, lamb, kmax)
    file_name = ff.replace_periods(file_name) + '.dat'

    # Load momentum (channel argument doesn't matter here)
    q_array, _ = vnn.load_momentum(kvnn, '3S1', kmax, kmid, ntot)
    
    # Initialize pair momentum distributions class for given potential
    snmd = single_nucleon_momentum_distributions(kvnn, channels, lamb, kmax, kmid, ntot)
    
    # Load r values and nucleonic densities (the r_array's are the same)
    r_array, rho_p_array = load_density(nucleus_name, 'proton', Z, N)
    r_array, rho_n_array = load_density(nucleus_name, 'neutron', Z, N)
    
    # Call LDA class
    lda = LDA(r_array, rho_p_array, rho_n_array)
    
    # Calculate nuclear-averaged momentum distributions
#     n_p_array = lda.local_density_approximation(q_array, snmd.n_lambda, 'p')
#     n_n_array = lda.local_density_approximation(q_array, snmd.n_lambda, 'n')
    # temporary
    n_p_array = lda.local_density_approximation(q_array, snmd.n_lambda_temp, 'p')
    n_n_array = lda.local_density_approximation(q_array, snmd.n_lambda_temp, 'n')
    
    # Open file and write header where we allocate roughly 18 centered spaces for each label
    f = open(data_directory + '/' + file_name, 'w')
    header = '#' + '{:^17s}{:^18s}{:^18s}'.format('q', 'proton', 'neutron')
    f.write(header + '\n')
    
    # Loop over momenta q
    for iq, q in enumerate(q_array):

        # Write to data file following the format from the header
        line = '{:^18.6f}{:^18.6e}{:^18.6e}'.format(q, n_p_array[iq], n_n_array[iq])
        f.write('\n' + line)

    # Close file
    f.close()
    
    # Save to Dropbox?
    if save_to_dropbox:
        _ = shutil.copy(data_directory + '/' + file_name, dropbox_directory)

In [13]:
# # LDA single-nucleon momentum distributions

# t0 = time.time() # Start time
# # Loop over nuclei
# for inucleus in nuclei:
#     lda_single_nucleon_momentum_distributions(inucleus, channels, kvnn, lamb, kmax, kmid, ntot,
#                                               save_to_dropbox)
# t1 = time.time() # Stop time
# mins = round( (t1 - t0) / 60.0, 5) # Minutes elapsed
# print('%.5f minutes elasped' % mins)

In [14]:
# # LDA single-nucleon momentum distributions with \lambda=1.5, 2, and 6 fm^-1

# lambdas = (1.5, 2.0, 6.0)

# t0 = time.time() # Start time
# # Loop over nuclei
# for inucleus in nuclei:
#     for ilamb in lambdas:
#         lda_single_nucleon_momentum_distributions(inucleus, channels, kvnn, ilamb, kmax, kmid, ntot,
#                                                   save_to_dropbox)
# t1 = time.time() # Stop time
# mins = round( (t1 - t0) / 60.0, 5) # Minutes elapsed
# print('%.5f minutes elasped' % mins)

__Deuteron pair momentum distribution $n_{\lambda}^d(q)$ under LDA__

In [15]:
def lda_deuteron_pair_momentum_distribution(kvnn, lamb, kmax=10.0, kmid=2.0, ntot=120, save_to_dropbox=False):
    """
    Deuteron-averaged and SRG-evolved pair momentum distributions n_\lambda^A(q).
    
    Parameters
    ----------
    kvnn : int
        This number specifies the potential.
    lamb : float
        SRG \lambda parameter [fm^-1].
    kmax : float, optional
        Maximum value in the momentum mesh [fm^-1].
    kmid : float, optional
        Mid-point value in the momentum mesh [fm^-1].
    ntot : int, optional
        Number of momentum points in mesh.
    save_to_dropbox : bool, optional
        Option to save file to Dropbox directory.

    """

    # Make file name
    file_name = 'lda_pmd_H2_channels_3S1_kvnn_%d_lamb_%.2f_kmax_%.1f' % (kvnn, lamb, kmax)
    file_name = ff.replace_periods(file_name) + '.dat'

    # Load momentum (channel argument doesn't matter here)
    q_array, _ = vnn.load_momentum(kvnn, '3S1', kmax, kmid, ntot)
    
    # Initialize deuteron momentum distributions class for given potential
    dmd = deuteron_momentum_distributions(kvnn, lamb, kmax, kmid, ntot)
    
    # Calculate pn pair momentum distribution
    n_d_array = dmd.local_density_approximation(q_array, 'pair')
    
    # Open file and write header where we allocate roughly 18 centered spaces for each label
    f = open(data_directory + '/' + file_name, 'w')
    header = '#' + '{:^17s}{:^18s}'.format('q', 'pn')
    f.write(header + '\n')
    
    # Loop over momenta q
    for iq, q in enumerate(q_array):

        # Write to data file following the format from the header
        line = '{:^18.6f}{:^18.6e}'.format(q, n_d_array[iq])
        f.write('\n' + line)

    # Close file
    f.close()
    
    # Save to Dropbox?
    if save_to_dropbox:
        _ = shutil.copy(data_directory + '/' + file_name, dropbox_directory)

In [16]:
# # LDA deuteron pair momentum distribution

# t0 = time.time() # Start time
# lda_deuteron_pair_momentum_distribution(kvnn, lamb, kmax, kmid, ntot, save_to_dropbox)
# t1 = time.time() # Stop time
# mins = round( (t1 - t0) / 60.0, 5) # Minutes elapsed
# print('%.5f minutes elasped' % mins)

In [17]:
# # LDA deuteron pair momentum distribution for AV18 with \lambda=1.5, 2, and 6 fm^-1

# lambdas = (1.5, 2.0, 6.0)

# t0 = time.time() # Start time
# for ilamb in lambdas:
#     lda_deuteron_pair_momentum_distribution(kvnn, ilamb, kmax, kmid, ntot, save_to_dropbox)
# t1 = time.time() # Stop time
# mins = round( (t1 - t0) / 60.0, 5) # Minutes elapsed
# print('%.5f minutes elasped' % mins)

__Deuteron single-nucleon momentum distribution $n_{\lambda}^d(q)$ under LDA__

In [18]:
def lda_deuteron_single_nucleon_momentum_distribution(kvnn, lamb, kmax=10.0, kmid=2.0, ntot=120,
                                                      save_to_dropbox=False):
    """
    Deuteron-averaged and SRG-evolved single-nucleon momentum distributions n_\lambda^A(q).
    
    Parameters
    ----------
    kvnn : int
        This number specifies the potential.
    lamb : float
        SRG \lambda parameter [fm^-1].
    kmax : float, optional
        Maximum value in the momentum mesh [fm^-1].
    kmid : float, optional
        Mid-point value in the momentum mesh [fm^-1].
    ntot : int, optional
        Number of momentum points in mesh.
    save_to_dropbox : bool, optional
        Option to save file to Dropbox directory.
        
    Notes
    -----
    Looks like there is a bug for last q-value (always zero).

    """

    # Make file name
    file_name = 'lda_snmd_H2_channels_3S1_kvnn_%d_lamb_%.2f_kmax_%.1f' % (kvnn, lamb, kmax)
    file_name = ff.replace_periods(file_name) + '.dat'

    # Load momentum (channel argument doesn't matter here)
    q_array, _ = vnn.load_momentum(kvnn, '3S1', kmax, kmid, ntot)
    
    # Initialize deuteron momentum distributions class for given potential
    dmd = deuteron_momentum_distributions(kvnn, lamb, kmax, kmid, ntot)
    
    # Calculate single-nucleon momentum distribution (proton and neutron give same result here)
    n_d_array = dmd.local_density_approximation(q_array, 'single-nucleon')

    # Open file and write header where we allocate roughly 18 centered spaces for each label
    f = open(data_directory + '/' + file_name, 'w')
    header = '#' + '{:^17s}{:^18s}'.format('q', 'single-nucleon')
    f.write(header + '\n')
    
    # Loop over momenta q
    for iq, q in enumerate(q_array):

        # Write to data file following the format from the header
        line = '{:^18.6f}{:^18.6e}'.format(q, n_d_array[iq])
        f.write('\n' + line)

    # Close file
    f.close()
    
    # Save to Dropbox?
    if save_to_dropbox:
        _ = shutil.copy(data_directory + '/' + file_name, dropbox_directory)

In [19]:
# # LDA deuteron single-nucleon momentum distribution

# t0 = time.time() # Start time
# lda_deuteron_single_nucleon_momentum_distribution(kvnn, lamb, kmax, kmid, ntot, save_to_dropbox)
# t1 = time.time() # Stop time
# mins = round( (t1 - t0) / 60.0, 5) # Minutes elapsed
# print('%.5f minutes elasped' % mins)

In [20]:
# # LDA deuteron single-nucleon momentum distribution with \lambda=1.5, 2, and 6 fm^-1

# lambdas = (1.5, 2.0, 6.0)

# t0 = time.time() # Start time
# for ilamb in lambdas:
#     lda_deuteron_single_nucleon_momentum_distribution(kvnn, ilamb, kmax, kmid, ntot, save_to_dropbox)
# t1 = time.time() # Stop time
# mins = round( (t1 - t0) / 60.0, 5) # Minutes elapsed
# print('%.5f minutes elasped' % mins)

__$a_2$ scaling factors__

In [21]:
def a2_scaling_factors(nuclei, channels, kvnn, lamb, integration_limits=(2.0,), kmax=10.0, kmid=2.0,
                       ntot=120, save_to_dropbox=False):
    """
    a_2 scaling factors evaluated using LDA nuclear-averaged, SRG-evolved single-nucleon momentum
    distributions. These factors are defined by
        ( \int_{q_start}^{q_stop} dq P^A(q) ) / ( \int_{q_start}^{q_stop} dq P^d(q) ),
    where P^A(q) is the single-nucleon probability distribution defined by P^A(q) = q^2 n^A(q) / A.
    The integration limits are set to 2-\infty fm^-1 by default. This function cannot be run without
    first generating data with lda_single_nucleon_momentum_distributions and
    lda_deuteron_single_nucleon_momentum_distribution.
    
    Parameters
    ----------
    nuclei : tuple
        Tuple that contains the details for various nuclei formatted as another tuple:
        (name (str), Z (int), N (int)) (e.g., nuclei[i] = ('O16', 8, 8)).
    channels : tuple
        Partial wave channels to include in the calculation (e.g., ('1S0', '3S1')).
    kvnn : int
        This number specifies the potential.
    lamb : float
        SRG \lambda parameter [fm^-1].
    integration_limits : tuple, optional
        Integration limits for a_2 calculation [fm^-1]. If no maximum is specified, then the function
        takes q to \infty.
    kmax : float, optional
        Maximum value in the momentum mesh [fm^-1].
    kmid : float, optional
        Mid-point value in the momentum mesh [fm^-1].
    ntot : int, optional
        Number of momentum points in mesh.
    save_to_dropbox : bool, optional
        Option to save file to Dropbox directory.
    
    """

    # Make file name
    file_name_1 = 'a2_scaling_factors_nuclei' # First part of name
    # Add each nucleus to file name
    for nucleus in nuclei:
        file_name_1 += '_%s' % nucleus[0]
    # Use this second part to load the snmd files easily which are common
    file_name_2 = '_channels'
    # Add each channel to file name
    for channel in channels:
        file_name_2 += '_%s' % channel
    file_name_2 += '_kvnn_%d_lamb_%.2f_kmax_%.1f' % (kvnn, lamb, kmax)
    # Add integration limits to file name
    if len(integration_limits) == 1: # Assumes only minimum specified
        file_name = file_name_1 + file_name_2 + '_limits_%.1f_%.1f' % (integration_limits[0], kmax)
    elif len(integration_limits) == 2:
        file_name = file_name_1 + file_name_2 + '_limits_%.1f_%.1f' % (integration_limits[0],
                                                                       integration_limits[1])

    # Replace periods
    file_name = ff.replace_periods(file_name) + '.dat'

    # Load momentum and weights
    q_array, q_weights = vnn.load_momentum(kvnn, '3S1', kmax, kmid, ntot)
    
    # Set integration limits for extraction of a_2
    if len(integration_limits) == 1:
        q_min_index = op.find_q_index(integration_limits[0], q_array)
        q_max_index = ntot # Corresponds to \infty
    elif len(integration_limits) == 2:
        q_min_index = op.find_q_index(integration_limits[0], q_array)
        q_max_index = op.find_q_index(integration_limits[1], q_array)
                
        
    # Load deuteron single-nucleon momentum distribution
    deuteron_file_name = 'lda_snmd_H2_channels_3S1_kvnn_%d_lamb_%.2f_kmax_%.1f' % (kvnn, lamb, kmax)
    deuteron_file_name = ff.replace_periods(deuteron_file_name) + '.dat'   
    deuteron_data = np.loadtxt(data_directory + '/' + deuteron_file_name)
    
    # Deuteron single-nucleon probability distribution (this is normalized to 1)
    p_d_array = q_array**2 * deuteron_data[:, 1]
    
    # Calculate deuteron denominator
    denominator = np.sum( (q_weights * p_d_array)[q_min_index:q_max_index] )
    
    # Open file and write header where we allocate roughly 18 centered spaces for each label
    f = open(data_directory + '/' + file_name, 'w')
    header = '#' + '{:^17s}{:^18s}'.format('A', 'a_2')
    f.write(header + '\n') 

    # Loop over nuclei
    for i, nucleus in enumerate(nuclei):

        # Details of nucleus
        nucleus_name = nucleus[0]
        A = nucleus[1] + nucleus[2]
        
        # Load nuclear-averaged single-nucleon momentum distribution
        nucleus_file_name = 'lda_snmd_%s' % nucleus_name + file_name_2
        nucleus_file_name = ff.replace_periods(nucleus_file_name) + '.dat'   
        nucleus_data = np.loadtxt(data_directory + '/' + nucleus_file_name)
        
        # Nuclear-averaged single-nucleon probability distribution for protons and neutrons
        p_p_array = q_array**2 * nucleus_data[:, 1] / A # proton
        p_n_array = q_array**2 * nucleus_data[:, 2] / A # neutron
        # Combine proton and neutron for single-nucleon probability distribution
        p_a_array = p_p_array + p_n_array
        
        # Calculate numerator
        numerator = np.sum( (q_weights * p_a_array)[q_min_index:q_max_index] )
        
        # Compute a_2
        a_2 = numerator / denominator

        # Write to data file following the format from the header
        line = '{:^18n}{:^18.6f}'.format(A, a_2)
        f.write('\n' + line)

    # Close file
    f.close()

    # Save to Dropbox?
    if save_to_dropbox:
        _ = shutil.copy(data_directory + '/' + file_name, dropbox_directory)

In [22]:
# # a_2 scaling factors (integration from 2 to \infty fm^-1)

# integration_limits = (2.0,)

# t0 = time.time() # Start time
# a2_scaling_factors(nuclei, channels, kvnn, lamb, integration_limits, kmax, kmid, ntot, save_to_dropbox)
# t1 = time.time() # Stop time
# mins = round( (t1 - t0) / 60.0, 5) # Minutes elapsed
# print('%.5f minutes elasped' % mins)

In [23]:
# # a_2 scaling factors (integration from 3.8 to 4.5 fm^-1)

# integration_limits = (3.8, 4.5)

# t0 = time.time() # Start time
# a2_scaling_factors(nuclei, channels, kvnn, lamb, integration_limits, kmax, kmid, ntot, save_to_dropbox)
# t1 = time.time() # Stop time
# mins = round( (t1 - t0) / 60.0, 5) # Minutes elapsed
# print('%.5f minutes elasped' % mins)

In [24]:
# # a_2 scaling factors with \lambda=1.5, 2, and 6 fm^-1 (doing both integration limits from before)

# lambdas = (1.5, 2.0, 6.0)
# integration_limits_tuple = ( (2.0,), (3.8, 4.5) )

# t0 = time.time() # Start time
# for ilamb in lambdas:
#     for iintegration_limits in integration_limits_tuple:
#         a2_scaling_factors(nuclei, channels, kvnn, ilamb, iintegration_limits, kmax, kmid, ntot,
#                            save_to_dropbox)
# t1 = time.time() # Stop time
# mins = round( (t1 - t0) / 60.0, 5) # Minutes elapsed
# print('%.5f minutes elasped' % mins)

__Extras__

In [25]:
def deuteron_partial_wave_ratios(kvnn, lamb, kmax=10.0, kmid=2.0, ntot=120, save_to_dropbox=False):
    """
    3S1-only / 3S1-3D1 ratio of the \delta U^2 SRG-evolved deuteron pair momentum distribution term
    using deuteron wave function (not LDA).
    
    Parameters
    ----------
    kvnn : int
        This number specifies the potential.
    lamb : float
        SRG \lambda parameter [fm^-1].
    kmax : float, optional
        Maximum value in the momentum mesh [fm^-1].
    kmid : float, optional
        Mid-point value in the momentum mesh [fm^-1].
    ntot : int, optional
        Number of momentum points in mesh.
    save_to_dropbox : bool, optional
        Option to save file to Dropbox directory.

    """

    # Load momentum
    q_array, _ = vnn.load_momentum(kvnn, '3S1', kmax, kmid, ntot)
    
    # Initialize deuteron momentum distributions class for given potential
    dmd = deuteron_momentum_distributions(kvnn, lamb, kmax, kmid, ntot)
    
    # Initialize file
    file_name = 'deuteron_partial_wave_ratios_kvnn_%d_lamb_%.2f_kmax_%.1f' % (kvnn, lamb, kmax)
    file_name = ff.replace_periods(file_name) + '.dat'
    f = open(data_directory + '/' + file_name, 'w')
    
    # Write header where we allocate roughly 18 centered spaces for each label
    header = '#' + '{:^17s}{:^18s}'.format('q', 'ratio')
    f.write(header + '\n')
    
    # Loop over momenta q
    for iq, q in enumerate(q_array):
        
        # Calculate ratio (don't need total)
        _, ratio = dmd.n_lambda_pair_exact(q, contributions='partial_wave_ratio')

        # Write to data file following the format from the header
        line = '{:^18.6f}{:^18.6e}'.format(q, ratio)
        f.write('\n' + line)

    # Close file
    f.close()
    
    # Save to Dropbox?
    if save_to_dropbox:
        _ = shutil.copy(data_directory + '/' + file_name, dropbox_directory)

In [26]:
# # Deuteron pair momentum distribution 3S1 / 3S1-3D1 ratios

# t0 = time.time() # Start time
# deuteron_partial_wave_ratios(kvnn, lamb, kmax, kmid, ntot, save_to_dropbox)
# t1 = time.time() # Stop time
# mins = round( (t1 - t0) / 60.0, 5) # Minutes elapsed
# print('%.5f minutes elasped' % mins)

In [27]:
def factorization_ratios(k_0, k_points, kvnn, channel, lamb, kmax=10.0, kmid=2.0, ntot=120,
                         save_to_dropbox=False):
    """
    Ratio of \delta U \delta U^{\dagger} with respect to momentum [fm^-1] for k = k_points in the numerator
    and k = k_0 in the denominator, and k' = q_array.

    Parameters
    ----------
    k_0 : float
        Momentum value for U(k_0, q) in the denominator [fm^-1].
    k_points : 1-D ndarray
        Array of momentum values for U(k_points, q) in the numerator [fm^-1].
    kvnn : int
        This number specifies the potential.
    channel : str
        The partial wave channel (e.g. '1S0').
    lamb : float
        SRG \lambda parameter [fm^-1].
    kmax : float, optional
        Maximum value in the momentum mesh [fm^-1].
    kmid : float, optional
        Mid-point value in the momentum mesh [fm^-1].
    ntot : int, optional
        Number of momentum points in mesh.
    save_to_dropbox : bool, optional
        Option to save file to Dropbox directory.

    """
    
    # Make file name
    file_name = 'factorization_ratios_k_0_%s_k_points' % ff.convert_number_to_string(k_0)
    # Add each k_i to file name
    for k_i in k_points:
        file_name += '_%s' % ff.convert_number_to_string(k_i)
    file_name += '_kvnn_%d_%s_lamb_%.2f_kmax_%.1f' % (kvnn, channel, lamb, kmax)
    file_name = ff.replace_periods(file_name) + '.dat'
    
    # Load momentum and weights
    q_array, q_weights = vnn.load_momentum(kvnn, channel, kmax, kmid, ntot)
    # Use factor_array to divide out momenta/weights for mesh-independent result
    if vnn.coupled_channel(channel):
        factor_array = np.concatenate( (np.sqrt(q_weights) * q_array, np.sqrt(q_weights) * q_array) ) * \
                       np.sqrt(2/np.pi)
    else:
        factor_array = np.sqrt(2*q_weights/np.pi) * q_array
    col, row = np.meshgrid(factor_array, factor_array)
    
    # Load initial and evolved Hamiltonians
    H_initial = vnn.load_hamiltonian(kvnn, channel, kmax, kmid, ntot)
    H_evolved = vnn.load_hamiltonian(kvnn, channel, kmax, kmid, ntot, method='srg', lamb=lamb)
    
    # Calculate SRG transformation
    U_matrix_unitless = SRG_unitary_transformation(H_initial, H_evolved)
    
    # Converting to units [fm^3]
    I_matrix_unitless = np.eye( len(factor_array), len(factor_array) )
    delta_U_matrix_unitless = U_matrix_unitless - I_matrix_unitless
    delta_U_matrix = delta_U_matrix_unitless / row / col
    
    # Calculate \delta U(k_0, q) \delta U^{\dagger}(q, k_0) array
    k_0_index = op.find_q_index(k_0, q_array)
    # Single channel (do not include coupled parts for coupled-channel)
    denominator_array = delta_U_matrix[k_0_index, :ntot] * delta_U_matrix.T[:ntot, k_0_index]
    
    # Store each ratio in dictionary
    d = {}
    
    # Loop over k_i in k_points
    for k_i in k_points:
        
        # Index of k_i in q_array
        k_i_index = op.find_q_index(k_i, q_array)
            
        # Obtain U(k_i, q) array
        numerator_array = delta_U_matrix[k_i_index, :ntot] * delta_U_matrix.T[:ntot, k_i_index]
        
        # Calculate ratio and store in dictionary with k_i as the key
        d[k_i] = abs(numerator_array / denominator_array)

    # Open file and write header where we allocate roughly 18 centered spaces for each label
    f = open(data_directory + '/' + file_name, 'w')
    
    # Header should show each momentum q and each k_i as column label
    header = '#' + '{:^17s}'.format('q')
    
    # Loop over k_i
    for k_i in k_points:
        
        # Add each k_i as column label
        header += '{:^18s}'.format( ff.convert_number_to_string(k_i) )

    f.write(header + '\n')
    
    # Loop over momenta q
    for iq, q in enumerate(q_array):
        
        line = '{:^18.6f}'.format(q)
        
        # Loop over each k_i and write each ratio
        for k_i in k_points:
            
            # Get ratio for given q and k_i
            ratio = d[k_i][iq] # k_i is a key and iq is an index here
            line += '{:^18.6e}'.format(ratio)
            
        f.write('\n' + line)

    # Close file
    f.close()
    
    # Save to Dropbox?
    if save_to_dropbox:
        _ = shutil.copy(data_directory + '/' + file_name, dropbox_directory)

In [28]:
# # Factorization ratios \delta U(k_i, q) \delta U^\dagger(q, k_i) / \delta U(k_0, q) \delta U^\dagger(q, k_0)

# # Do k_0 = 0.1 fm^-1
# k_0 = 0.1
# k_points = np.array([0.5, 1.0, 1.5, 3.0])
# channel = '3S1'

# t0 = time.time() # Start time
# factorization_ratios(k_0, k_points, kvnn, channel, lamb, kmax, kmid, ntot, save_to_dropbox)
# t1 = time.time() # Stop time
# mins = round( (t1 - t0) / 60.0, 5) # Minutes elapsed
# print('%.5f minutes elasped' % mins)

In [29]:
def factorization_ratios_k_dependence(q_0, q_points, kvnn, channel, lamb, kmax=10.0, kmid=2.0, ntot=120,
                                       save_to_dropbox=False):
    """
    Ratio of \delta U \delta U^{\dagger} with respect to momentum [fm^-1] for q = q_points in the numerator
    and q = q_0 in the denominator, and k = k_array. This is very similar to factorization_ratios but shows
    the dependence on low-k instead.

    Parameters
    ----------
    q_0 : float
        Momentum value for U(k, q_0) in the denominator [fm^-1].
    q_points : 1-D ndarray
        Array of momentum values for U(k, q_points) in the numerator [fm^-1].
    kvnn : int
        This number specifies the potential.
    channel : str
        The partial wave channel (e.g. '1S0').
    lamb : float
        SRG \lambda parameter [fm^-1].
    kmax : float, optional
        Maximum value in the momentum mesh [fm^-1].
    kmid : float, optional
        Mid-point value in the momentum mesh [fm^-1].
    ntot : int, optional
        Number of momentum points in mesh.
    save_to_dropbox : bool, optional
        Option to save file to Dropbox directory.

    """
    
    # Make file name
    file_name = 'factorization_ratios_q_0_%s_q_points' % ff.convert_number_to_string(q_0)
    # Add each k_i to file name
    for q_i in q_points:
        file_name += '_%s' % ff.convert_number_to_string(q_i)
    file_name += '_kvnn_%d_%s_lamb_%.2f_kmax_%.1f' % (kvnn, channel, lamb, kmax)
    file_name = ff.replace_periods(file_name) + '.dat'
    
    # Load momentum and weights
    k_array, k_weights = vnn.load_momentum(kvnn, channel, kmax, kmid, ntot)
    # Use factor_array to divide out momenta/weights for mesh-independent result
    if vnn.coupled_channel(channel):
        factor_array = np.concatenate( (np.sqrt(k_weights) * k_array, np.sqrt(k_weights) * k_array) ) * \
                       np.sqrt(2/np.pi)
    else:
        factor_array = np.sqrt(2*k_weights/np.pi) * k_array
    col, row = np.meshgrid(factor_array, factor_array)
    
    # Load initial and evolved Hamiltonians
    H_initial = vnn.load_hamiltonian(kvnn, channel, kmax, kmid, ntot)
    H_evolved = vnn.load_hamiltonian(kvnn, channel, kmax, kmid, ntot, method='srg', lamb=lamb)
    
    # Calculate SRG transformation
    U_matrix_unitless = SRG_unitary_transformation(H_initial, H_evolved)
    
    # Converting to units [fm^3]
    I_matrix_unitless = np.eye( len(factor_array), len(factor_array) )
    delta_U_matrix_unitless = U_matrix_unitless - I_matrix_unitless
    delta_U_matrix = delta_U_matrix_unitless / row / col
    
    # Calculate \delta U(k, q_0) \delta U^{\dagger}(q_0, k) array
    q_0_index = op.find_q_index(q_0, k_array)
    # Single channel (do not include coupled parts for coupled-channel)
    denominator_array = delta_U_matrix[:ntot, q_0_index] * delta_U_matrix.T[q_0_index, :ntot]
    
    # Store each ratio in dictionary
    d = {}
    
    # Loop over q_i in q_points
    for q_i in q_points:
        
        # Index of q_i in k_array
        q_i_index = op.find_q_index(q_i, k_array)
            
        # Obtain U(k_i, q) array
        numerator_array = delta_U_matrix[:ntot, q_i_index] * delta_U_matrix.T[q_i_index, :ntot]
        
        # Calculate ratio and store in dictionary with k_i as the key
        d[q_i] = abs(numerator_array / denominator_array)

    # Open file and write header where we allocate roughly 18 centered spaces for each label
    f = open(data_directory + '/' + file_name, 'w')
    
    # Header should show each momentum k and each q_i as column label
    header = '#' + '{:^18s}'.format('k')
    
    # Loop over k_i
    for q_i in q_points:
        
        # Add each q_i as column label
        header += '{:^18s}'.format( ff.convert_number_to_string(q_i) )

    f.write(header + '\n')
    
    # Loop over momenta q
    for ik, k in enumerate(k_array):
        
        line = '{:^18.6f}'.format(k)
        
        # Loop over each q_i and write each ratio
        for q_i in q_points:
            
            # Get ratio for given k and q_i
            ratio = d[q_i][ik] # q_i is a key and ik is an index here
            line += '{:^18.6e}'.format(ratio)
            
        f.write('\n' + line)

    # Close file
    f.close()
    
    # Save to Dropbox?
    if save_to_dropbox:
        _ = shutil.copy(data_directory + '/' + file_name, dropbox_directory)

In [30]:
# # Factorization ratios \delta U(k, q_i) \delta U^\dagger(q_i, k) / \delta U(k, q_0) \delta U^\dagger(q_0, k)

# # Do q_0 = 6.0 fm^-1
# q_0 = 6.0
# q_points = np.array([1.0, 2.0, 3.0, 4.0])
# channel = '3S1'

# t0 = time.time() # Start time
# factorization_ratios_k_dependence(q_0, q_points, kvnn, channel, lamb, kmax, kmid, ntot, save_to_dropbox)
# t1 = time.time() # Stop time
# mins = round( (t1 - t0) / 60.0, 5) # Minutes elapsed
# print('%.5f minutes elasped' % mins)

In [31]:
def snmd_partial_wave_contributions(nuclei, channels, kvnn, lamb, kmax=10.0, kmid=2.0, ntot=120, 
                                    save_to_dropbox=False):
    """
    Fraction of partial wave contributions to total normalization of the high-q tail in nuclear-averaged
    and SRG-evolved single-nucleon momentum distributions n_\lambda^A(q). This is done by computing
        \int_2^\infty dq q^2 < n_A^p(q) >_X / \int_2^\infty dq q^2 < n_A^p(q) >,
    where X is a single partial wave channel. (Overall factor in front of integrals do not matter here.)
    
    Parameters
    ----------
    nuclei : tuple
        Tuple that contains the details for various nuclei formatted as another tuple:
        (name (str), Z (int), N (int)) (e.g., nuclei[i] = ('O16', 8, 8)).
    channels : tuple
        Partial wave channels to compute (e.g., ('1S0', '3S1', '3P0', '1P1', '3P1')).
    kvnn : int
        This number specifies the potential.
    lamb : float
        SRG \lambda parameter [fm^-1].
    kmax : float, optional
        Maximum value in the momentum mesh [fm^-1].
    kmid : float, optional
        Mid-point value in the momentum mesh [fm^-1].
    ntot : int, optional
        Number of momentum points in mesh.
    save_to_dropbox : bool, optional
        Option to save file to Dropbox directory.
        
    """
    
    # Normalization factor (for printing purposes - divides out in ratio)
    factor = 4*np.pi / (2*np.pi)**3
    
    # Make file name
    file_name = 'snmd_partial_wave_contributions_nuclei'
    # Add each nucleus to file name
    for nucleus in nuclei:
        file_name += '_%s' % nucleus[0]
    file_name += '_channels'
    # Add each channel to file name
    for channel in channels:
        file_name += '_%s' % channel
    file_name += '_kvnn_%d_lamb_%.2f_kmax_%.1f' % (kvnn, lamb, kmax)
    file_name = ff.replace_periods(file_name) + '.dat'
    
    # Load momentum (channel argument doesn't matter here)
    q_array, q_weights = vnn.load_momentum(kvnn, '1S0', kmax, kmid, ntot)
    
    # Set integration limits
    q_min = op.find_q_index(2.0, q_array)
    
    # Calculate total proton momentum distributions first
    
    # Initialize pair momentum distributions class for given potential and all partial waves
    snmd_all = single_nucleon_momentum_distributions(kvnn, channels, lamb, kmax, kmid, ntot)
    
    # Store total proton momentum distribution for each nucleus in dictionary
    d = {}
    for nucleus in nuclei:
        
        # Name of nucleus, proton number, neutron number, and mass number
        nucleus_name = nucleus[0]
        Z = nucleus[1]
        N = nucleus[2]
        A = N + Z
        
        # Make nucleus name the key
        d[nucleus_name] = {}

        # Load r values and nucleonic densities (the r_array's are the same)
        r_array, rho_p_array = load_density(nucleus_name, 'proton', Z, N)
        r_array, rho_n_array = load_density(nucleus_name, 'neutron', Z, N)
    
        # Call LDA class
        lda = LDA(r_array, rho_p_array, rho_n_array)
    
        # Calculate nuclear-averaged proton momentum distribution for all partial waves
        d[nucleus_name]['total'] = lda.local_density_approximation(q_array, snmd_all.n_lambda_temp, 'p')
        
        # Loop over each partial wave channel and calculate contribution
        for channel in channels:
            
            # This makes it so you can loop over the single channel (otherwise snmd.py wouldn't work)
            channel_tuple = (channel,)
            
            # Initialize pair momentum distributions class for given potential and one partial wave
            snmd_partial = single_nucleon_momentum_distributions(kvnn, channel_tuple, lamb, kmax, kmid, ntot)
            
            # Calculate nuclear-averaged proton momentum distribution
            # d[nucleus_name][channel] = lda.local_density_approximation(q_array, snmd_partial.n_lambda, 'p')
            # temporary
            d[nucleus_name][channel] = lda.local_density_approximation(q_array, snmd_partial.n_lambda_temp, 'p')

    # Open file and write header where we allocate roughly 18 centered spaces for each label
    f = open(data_directory + '/' + file_name, 'w')
    
    # Header should show nuclei
    header = '{:^18s}'.format('   ')
    
    # Loop over nuclei
    for nucleus in nuclei:
        
        # Add each nucleus as a column label
        header += '{:^18s}'.format(nucleus[0])
    
    # Write header
    f.write(header + '\n')
        
    # Loop over partial wave channels
    for channel in channels:
        
        # Add each channel as a row label
        line = '{:^18s}'.format(channel)
        
        # Loop over nuclei to write to each column
        for nucleus in nuclei:
            
            # Compute partial wave contribution to tail normalization (without 4\pi/(2\pi)^3)
            numerator = np.sum( ( q_weights * q_array**2 * d[ nucleus[0] ][channel] )[q_min:] )
            
            # Compute total tail normalization (without 4\pi/(2\pi)^3)
            denominator = np.sum( ( q_weights * q_array**2 * d[ nucleus[0] ]['total'] )[q_min:] )
    
            # Add fraction to row
            line += '{:^18e}'.format(numerator/denominator)
            
        # Write row
        f.write(line + '\n')
        
    # Close file
    f.close()
    
    # Save to Dropbox?
    if save_to_dropbox:
        _ = shutil.copy(data_directory + '/' + file_name, dropbox_directory)

In [ ]:
# Partial wave contributions to tail of single-nucleon momentum distribution

t0 = time.time() # Start time
snmd_partial_wave_contributions(nuclei, ('1S0', '3S1', '3P0', '1P1', '3P1'), kvnn, lamb, kmax, kmid, ntot,
                                save_to_dropbox)
t1 = time.time() # Stop time
mins = round( (t1 - t0) / 60.0, 5) # Minutes elapsed
print('%.5f minutes elasped' % mins)

In [ ]:
# contacts
# function

In [ ]:
# write for av18